In [1]:
# This code calculates the vertical DEM shift using the Nuth and Kääb point cloud coregistration
# activate the xdem_dev environment

import geoutils as gu
import numpy as np
import pandas as pd

import xdem

/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/skgstat/models.py:22: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/skgstat/models.py:86: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site

In [17]:
# **Inputs**
icesat_file = 'Sites/RCEW/DEMs/RCEW-ICESat2-ATL03'
CoReg_file = 'Sites/RCEW/DEMs/RCEW_1m_WGS84UTM11_WGS84'
DEM_file = 'Sites/RCEW/DEMs/RCEW_1m_WGS84UTM11_WGS84'
slope_file = 'Sites/RCEW/DEMs/RCEW_1m_WGS84UTM11_WGS84-slope'
aspect_file = 'Sites/RCEW/DEMs/RCEW_1m_WGS84UTM11_WGS84-aspect'

In [18]:
# **files**
reference_dem = xdem.DEM(icesat_file + '.tif')
dem_to_be_aligned = xdem.DEM(CoReg_file + '.tif')
dem_elev = xdem.DEM(DEM_file + '.tif')
dem_slope = xdem.DEM(slope_file + '.tif')
dem_aspect = xdem.DEM(aspect_file + '.tif')

/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/geoutils/raster/raster.py:1407: UserWarning: Setting default nodata -99999 to mask non-finite values found in the array, as no nodata value was defined.
  warnings.warn(
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/geoutils/raster/raster.py:1407: UserWarning: Setting default nodata -99999 to mask non-finite values found in the array, as no nodata value was defined.
  warnings.warn(
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/geoutils/raster/raster.py:1407: UserWarning: Setting default nodata -99999 to mask non-finite values found in the array, as no nodata value was defined.
  warnings.warn(
/Users/karinazikan/opt/anaconda3/envs/xdem-dev/lib/python3.11/site-packages/geoutils/raster/raster.py:1407: UserWarning: Setting default nodata -99999 to mask non-finite values found in the array, as no nodata value was defined.
  warnings.warn(
/Users/karinazik

In [ ]:
# **difference before coregistration**
diff_before = reference_dem - dem_to_be_aligned
diff_before.show(cmap="coolwarm_r", vmin=-1, vmax=1, cbar_title="Elevation change (m)")

In [19]:
# **Nuth and Kääb coregistration**
nuth_kaab = xdem.coreg.NuthKaab()

nuth_kaab.fit(reference_dem, dem_to_be_aligned)

In [20]:
# **apply coregistration shift**
aligned_dem = nuth_kaab.apply(dem_elev)
aligned_slope = nuth_kaab.apply(dem_slope)
aligned_aspect = nuth_kaab.apply(dem_aspect)

In [21]:
# **export coregistered DEMs**
aligned_dem.save(DEM_file +'_CoReg.tif')
aligned_slope.save(slope_file +'_CoReg.tif')
aligned_aspect.save(aspect_file +'_CoReg.tif')